# SDT

Soft Decision Trees are trained with mini-batch gradient descent, where each inner node `i` has a learned filter $w_{i}$ and bias $b_{i}$, and each leaf node `l` has a learned distribution $Q_{l}$. At each node the probability of taking rightmost branch is: $$p_{i}(x) = \sigma(xw_{i} + b_{i})$$

where `x` is the input to the model and $\sigma$ is the sigmoid logistic function.

![sdt](images/sdt.png)

In [ ]:
# default_exp model

In [ ]:
#hide
IN_COLAB = 'google.colab' in str(get_ipython())
if IN_COLAB:
  !pip3 install -Uqq fastbook

In [ ]:
#hide
if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#hide
if IN_COLAB:
  from pathlib import Path
  from nbdev.imports import Config
  project_path = Path('/content/drive/My Drive/Colab Notebooks/github/sdt')
  get_ipython().magic(f'cd {project_path}')
  get_ipython().magic(f'cd {Config().nbs_path}')

/content/drive/My Drive/Colab Notebooks/github/sdt
/content/drive/My Drive/Colab Notebooks/github/sdt


In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from fastai.vision.all import *

## Tree

In [ ]:
#export
class Tree(Module):
  def __init__(self, input_dim=784, output_dim=10, type_='inner'):
    super().__init__()

    self.left  = None
    self.right = None
    self.type_ = type_
    
    self.input_dim  = input_dim
    self.output_dim = output_dim 

    self.node_params()
  
  def node_params(self):
    if self.type_ == 'inner': self.inner_node_params()
    else: self.leaf_node_params()
  
  def forward(self, x):
    if self.type_ == 'inner': return self.inner_node_forward(x)
    else: return self.leaf_node_forward(x)
  
  def inner_node_params(self): 
    self.beta = nn.Parameter(torch.randn(1))
    self.fc   = nn.Linear(self.input_dim, 1)
    self.sigmoid = nn.Sigmoid()
  
  def leaf_node_params(self) : self.softmax, self.param = nn.Softmax(dim=1), nn.Parameter(torch.randn(self.output_dim))

  def inner_node_forward(self, x): return self.sigmoid(self.beta*self.fc(x))
  def leaf_node_forward(self, x) : return self.softmax(self.param.view(1, -1)).expand(x.shape[0], self.output_dim)

In [ ]:
#export
def sdt_init(current_depth, depth, tree, input_dim, output_dim):
  if current_depth == depth - 1:
    tree.left  = Tree(input_dim=input_dim, output_dim=output_dim, type_='leaf')
    tree.right = Tree(input_dim=input_dim, output_dim=output_dim, type_='leaf') 
  else:
    tree.left = Tree(input_dim=input_dim, output_dim=output_dim)
    sdt_init(current_depth + 1, depth, tree.left, input_dim, output_dim)
    tree.right = Tree(input_dim=input_dim, output_dim=output_dim)
    sdt_init(current_depth + 1, depth, tree.right, input_dim, output_dim)
  
  return tree

In [ ]:
#export
class SDT(Module):
  def __init__(self, tree, n_leaves, n_classes):
    super().__init__()
    
    self.numers    = []
    self.denoms    = []
    self.lambdas   = []
    self.path_prob = []
    self.outs      = []

    self.tree      = tree
    self.n_leaves  = n_leaves
    self.n_inner_nodes = n_leaves - 1
    self.n_classes = n_classes
  
  def run(self, x, tree, prob, current_depth, index):
    if tree.type_ == 'leaf':
      self.path_prob.append(prob)
      self.outs.append(tree.forward(x))
    else:
      left_prob = tree.forward(x)

      self.numers.append(prob * left_prob)
      self.denoms.append(prob)
      
      leaf_prob = self.run(x, 
                           tree.left, 
                           prob * left_prob, 
                           current_depth + 1, 
                           index + 1)
      
      right_prob = 1 - left_prob
      leaf_prob  = self.run(x, 
                            tree.right, 
                            prob * right_prob, 
                            current_depth + 1, 
                            index + 2)

  def forward(self, x):
    x = x.view(x.shape[0], -1)
    self.prob = torch.ones_like(x)[:, 0:1]
    
    self.run(x, self.tree, self.prob, current_depth=0, index=0)

    self.path_prob  = torch.cat(self.path_prob).view(self.n_leaves, x.shape[0])\
                      .permute(1, 0)
    self.outs       = torch.cat(self.outs).view(self.n_leaves, x.shape[0], self.n_classes)\
                      .permute(1, 0, 2)
    
    self.numers     = torch.cat(self.numers).view(self.n_inner_nodes, x.shape[0])\
                           .permute(1, 0)
    self.denoms     = torch.cat(self.denoms).view(self.n_inner_nodes, x.shape[0])\
                           .permute(1, 0)
    
    return self.path_prob, self.outs

  @property
  def get_path_prob(self): return self.path_prob
  @property
  def get_outs(self)     : return self.outs
  @property
  def get_numers(self)   : return self.numers
  @property
  def get_denoms(self)   : return self.denoms

  def reset(self):
    self.path_prob = []
    self.outs = []
    self.numers = []
    self.denoms = []

##Export

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_data.ipynb.
Converted 01_model.ipynb.
Converted 02_loss.ipynb.
Converted 03_train.ipynb.
Converted index.ipynb.
